In [1]:
# French Baby Names Map Visualization
import altair as alt
import pandas as pd
import geopandas as gpd

alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
2060819,2,CARMEN,1934,86,3
3495023,2,SABRINA,2006,37,3
1686093,1,YANNICK,1983,82,6
1316388,1,PATRICE,1971,70,9
2179986,2,CLAUDE,1938,79,13


In [12]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
81,81,Tarn,"POLYGON ((1.99017 44.14945, 2.0297 44.15704, 2..."
30,30,Gard,"POLYGON ((3.37365 44.17076, 3.43083 44.148, 3...."
84,84,Vaucluse,"MULTIPOLYGON (((4.89291 44.36482, 4.90663 44.3..."
31,31,Haute-Garonne,"POLYGON ((0.95398 43.78737, 0.9778 43.78644, 0..."
76,76,Seine-Maritime,"POLYGON ((1.38155 50.06577, 1.40926 50.05707, ..."


In [13]:
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

In [45]:
chosen_name = 'Camille'
chosen_department = 'All'

names['nombre'] = pd.to_numeric(names['nombre'], errors='coerce')
names['annais'] = pd.to_numeric(names['annais'], errors='coerce')

df = names.dropna(subset=['nombre', 'annais'])

df_name = df[df['preusuel'].str.upper() == chosen_name.upper()]

if chosen_department != 'All':
    df_name = df_name[df['nom'].str.upper() == chosen_department.upper()]

# grouped = df_name.groupby(['annais', 'sexe'])['nombre'].sum().reset_index()

# pivot = grouped.pivot(index='annais', columns='sexe', values='nombre').fillna(0)
# pivot.columns = ['Boys', 'Girls']

# pivot['Total'] = pivot['Girls'] + pivot['Boys']
# pivot['% Girls'] = (pivot['Girls'] / pivot['Total']) * 100
# pivot['% Boys'] = (pivot['Boys'] / pivot['Total']) * 100

# pivot['Year'] = pivot.index  # optional if index is unnamed
# plot_df = pivot[['Year', 'Girls', 'Boys', '% Girls', '% Boys']].melt(
#     id_vars=['Year', 'Girls', 'Boys'],
#     var_name='Sex',
#     value_name='Percentage'
# )

# plot_df = pivot[['% Girls', '% Boys']].reset_index().melt(id_vars='annais', 
#                                                           var_name='Sex', 
#                                                           value_name='Percentage')

# plot_df['Sex'] = plot_df['Sex'].map({'% Girls': 'Girls', '% Boys': 'Boys'})

grouped = df_name.groupby(['annais', 'sexe'])['nombre'].sum().reset_index()

pivot = grouped.pivot(index='annais', columns='sexe', values='nombre').fillna(0)
pivot.columns = ['Boys', 'Girls'] 

pivot['Total'] = pivot['Girls'] + pivot['Boys']
pivot['% Girls'] = (pivot['Girls'] / pivot['Total']) * 100
pivot['% Boys'] = (pivot['Boys'] / pivot['Total']) * 100

pivot = pivot.reset_index() 
pivot = pivot.rename(columns={'annais': 'Year'}) 

plot_df = pivot[['Year', 'Girls', 'Boys', '% Girls', '% Boys']].melt(
    id_vars=['Year', 'Girls', 'Boys'],
    var_name='Sex',
    value_name='Percentage'
)

plot_df['Sex'] = plot_df['Sex'].map({'% Girls': 'Girls', '% Boys': 'Boys'})


if chosen_department == 'All':
    title_str = f"Percentage of Boys and Girls Named {chosen_name} Over Time in France"
else:
    title_str = f"Percentage of Boys and Girls Named {chosen_name} Over Time in {chosen_department}"

# chart = alt.Chart(plot_df).mark_line(point=True).encode(
#     # x=alt.X('annais:O', title='Year'),
#     x=alt.X('annais:O', title='Year', axis=alt.Axis(labelExpr="datum.value % 5 === 0 ? datum.value : ''")),
#     y=alt.Y('Percentage:Q', title='Percentage (%)'),
#     color='Sex:N'
# ).properties(
#     title=title_str,
#     width=600,
#     height=400
# )
chart = alt.Chart(plot_df).mark_line(point=True).encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelExpr="datum.value % 10 === 0 ? datum.value : ''")),
    y=alt.Y('Percentage:Q', title='Percentage (%)'),
    color='Sex:N',
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Sex:N', title='Sex'),
        alt.Tooltip('Percentage:Q', format=".2f", title='% of total'),
        alt.Tooltip('Girls:Q', title='Girls (abs)'),
        alt.Tooltip('Boys:Q', title='Boys (abs)')
    ]
).properties(
    title=title_str,
    width=600,
    height=400
)


chart


alt.Chart(...)

In [ ]:
# TODO 
# 1. X-axis every 5-10 years if a lot - Done
# 2. No zoom - Done
# 3. Total number of names if hover over a point - Done
# 4. Combination of names (a chosen pair of names)